In [1]:
import glob
import numpy as np 
import time
import math
import random
from scipy import linalg as LA
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.svm import NuSVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron
import statsmodels.api as sm
from autograd import grad
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from termcolor import colored
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import optimize
from sklearn.svm import LinearSVC
from termcolor import colored
from datetime import datetime
import os
import similaritymeasures
import tslearn
from tslearn.metrics import dtw
from scipy.spatial.distance import directed_hausdorff
import trjtrypy as tt
from trjtrypy.distances import d_Q
from trjtrypy.distances import d_Q_pi
import trjtrypy.visualizations as vs
from scipy.spatial import distance
from trjtrypy.featureMappings import curve2vec

In [2]:
I = glob.glob('T-Drive/**/*.txt', recursive=True)

In [3]:
# runtime about 35s
Start_time = time.time()
J = [0] * len(I)

for i in range(len(I)):
    for j in range(len(I)):
        if int(I[j][28:-4]) == i+1:
            J[i] = I[j]
print('total time =', time.time() - Start_time)

total time = 37.63029599189758


In [4]:
taxi_id = np.array([J[i][28:-4] for i in range(len(J))])
I = np.sort(list(map(int, taxi_id)))
True in I == list(map(int, taxi_id)) # so the taxi ids are nn.arange(len(J))

array([ True,  True,  True, ...,  True,  True,  True])

In [5]:
len(J)

10357

In [6]:
# create 'daysDate' function to convert start and end time to a float number of days

def days_date(time_str):
    date_format = "%Y-%m-%d %H:%M:%S"
    current = datetime.strptime(time_str, date_format)
    date_format = "%Y-%m-%d"
    bench = datetime.strptime('2006-12-30', date_format)
    no_days = current - bench
    delta_time_days = no_days.days + current.hour / 24.0 + current.minute / (24. * 60.) + current.second / (24. * 3600.)
    return delta_time_days

days_date = np.vectorize(days_date)
float1 = np.vectorize(float)

In [7]:
def read_file(file_name):
    data = []
    f = open(file_name, "r")
    for line in f:
        item = line.strip().split(",")
        if len(item) ==4:
            data.append(np.asarray(item[1:]))
    data = np.array(data)
    return data

## Remove stationary points

In [8]:
def remove_segments(traj): # removes stationary points
    p2 = traj[:,1:][1:]
    p1 = traj[:,1:][:-1]
    L = ((p2-p1)*(p2-p1)).sum(axis =1)
    I = np.where(L>1e-16)[0]
    return traj[I]

### In data1 below:
    (1) there is no stationary points
    (2) there is no trajectories with less than 1 or more than 200 waypoints

In [9]:
# runtime about 350s
Start_time = time.time()
T = len(J)
data1 = [] 
taxi_ids = []
for i in range(T):
    a = read_file(J[i])
    if len(a) > 1000:  
        a[:,0] = days_date(a[:,0])
        a = float1(a)
        data1.append(remove_segments(a)) 
        taxi_ids.append(i)

taxi_ids = np.array(taxi_ids)   
data1 = np.array(data1, dtype='object')
I = np.where(np.array([len(data1[i]) for i in range(len(data1))]) > 0)[0]
data1 = data1[I]
taxi_ids = np.array(taxi_ids[I], dtype='object')
print('total time =', time.time() - Start_time)
print("len(data1) =", len(data1))
print("selected taxi_ids: \n", taxi_ids)

total time = 352.83437275886536
len(data1) = 6611
selected taxi_ids: 
 [1 2 9 ... 10339 10350 10353]


In [10]:
def partition(trajectory, threshold=20):
    trajectories = []
    a = 24 * 60 * (trajectory[-1][0] - trajectory[0][0])
    if a <= threshold:
        return np.array(trajectory.reshape(1, len(trajectory), 3))
    else: 
        i = 0
        while a > threshold:
            j = i + 0
            val = 0
            while val < threshold: 
                if i < len(trajectory) - 1:
                    temp = val + 0
                    val += 24 * 60 * (trajectory[:,0][1:][i] - trajectory[:,0][:-1][i])
                    i += 1
                else: 
                    break
            if len(trajectory[j:i-1]) > 0:
                trajectories.append(trajectory[j:i-1])
            a = a - val
        if len(trajectory[i:]) > 0:
            trajectories.append(trajectory[i:])
    trajectories = np.array(trajectories, dtype='object')
    return trajectories

In [11]:
# Check to see if partitioning into less than 20 minutes worked correctly
for j in range(len(data1)):
    A = partition(data1[j], threshold=20)
    B = np.array([24 * 60 * (A[i][-1][0] - A[i][0][0]) for i in range(len(A))], dtype='object')
    I = np.where(B > 800)[0]
    if len(I) > 0: 
        print(j)

### In data3 below 
    1) any taxi has between 100 and 200 trajectories
    2) in each taxi dataset any trajectory has less than 20 minutes time

In [12]:
Start_time = time.time()
data3 = []
idxs = []
for i in range(len(data1)):
    D = []
    A = partition(data1[i], threshold=20)
    for j in range(len(A)):
        if ((len(A[j]) >= 10) and (len(A[j]) <= 200)):
            D.append(A[j])
    if (len(D) >= 100 and len(D) <= 200):
        data3.append(np.array(D, dtype='object'))
        idxs.append(i)

taxi_idxs = taxi_ids[idxs]
data3 = np.array(data3, dtype='object')
taxi_idxs = np.array(taxi_idxs)

print('total time =', time.time() - Start_time)
print("len(data3) =", len(data3))
print("len(taxi_idxs) =", len(taxi_idxs))

A = np.array([len(data3[i]) for i in range(len(data3))])
print(colored(f"min and max length of selected users: {min(A), max(A)}", "green"))

print("number of selected users:", len(data3))
print("idxs: \n", idxs)
print(colored(f"selected taxi ids: \n {taxi_idxs}", "blue"))

total time = 26.36145806312561
len(data3) = 433
len(taxi_idxs) = 433
min and max length of selected users: (100, 200)
number of selected users: 433
idxs: 
 [2, 4, 16, 17, 18, 75, 76, 97, 137, 333, 504, 505, 542, 558, 559, 561, 571, 572, 573, 587, 595, 597, 601, 612, 620, 636, 667, 727, 753, 897, 910, 923, 926, 933, 939, 940, 942, 945, 947, 972, 989, 1007, 1034, 1036, 1047, 1060, 1063, 1066, 1067, 1069, 1076, 1161, 1174, 1181, 1189, 1190, 1195, 1197, 1208, 1210, 1212, 1215, 1217, 1218, 1224, 1229, 1231, 1240, 1244, 1246, 1249, 1250, 1257, 1269, 1280, 1292, 1294, 1299, 1300, 1304, 1307, 1310, 1405, 1421, 1422, 1423, 1429, 1435, 1439, 1440, 1443, 1469, 1477, 1488, 1490, 1499, 1519, 1531, 1540, 1544, 1551, 1562, 1618, 1622, 1642, 1643, 1670, 1674, 1676, 1677, 1679, 1680, 1690, 1701, 1702, 1708, 1717, 1745, 1847, 1849, 1855, 1858, 1859, 1860, 1865, 1881, 1882, 1883, 1894, 1895, 1897, 1902, 1907, 1914, 1929, 2025, 2059, 2070, 2126, 2127, 2237, 2299, 2300, 2349, 2364, 2374, 2375, 2378, 2437, 

In [13]:
taxi_idxs + 1

array([10, 16, 31, 34, 35, 131, 132, 177, 251, 490, 750, 751, 799, 817,
       818, 820, 839, 841, 842, 861, 875, 877, 883, 903, 919, 947, 995,
       1081, 1112, 1309, 1327, 1350, 1353, 1360, 1367, 1370, 1372, 1375,
       1378, 1415, 1439, 1466, 1511, 1513, 1526, 1557, 1563, 1566, 1567,
       1570, 1580, 1695, 1712, 1725, 1737, 1738, 1745, 1747, 1762, 1768,
       1770, 1773, 1775, 1776, 1789, 1794, 1796, 1817, 1825, 1830, 1837,
       1838, 1847, 1876, 1909, 1935, 1938, 1948, 1950, 1958, 1965, 1971,
       2108, 2126, 2127, 2128, 2138, 2146, 2153, 2154, 2157, 2193, 2205,
       2222, 2226, 2240, 2274, 2295, 2313, 2318, 2331, 2353, 2448, 2454,
       2486, 2487, 2544, 2550, 2553, 2554, 2566, 2567, 2580, 2593, 2594,
       2605, 2617, 2672, 2862, 2866, 2876, 2880, 2881, 2883, 2888, 2907,
       2909, 2910, 2924, 2925, 2927, 2938, 2943, 2955, 2975, 3106, 3142,
       3154, 3259, 3260, 3425, 3511, 3512, 3593, 3622, 3644, 3647, 3655,
       3772, 3801, 3866, 3872, 3896, 3915, 3918, 3938

In [14]:
data = data3
Min = np.min([np.min([np.min(data[i][j][:,1:], 0) for j in range(len(data[i]))], 0) for i in range(len(data))], 0)
Max = np.max([np.max([np.max(data[i][j][:,1:], 0) for j in range(len(data[i]))], 0) for i in range(len(data))], 0)
Mean = np.mean([np.mean([np.mean(data[i][j][:,1:], 0) for j in range(len(data[i]))], 0) for i in range(len(data))], 0)
Std = np.std([np.std([np.std(data[i][j][:,1:], 0) for j in range(len(data[i]))], 0) for i in range(len(data))], 0)
Min, Mean, Max, Std

(array([0., 0.]),
 array([116.40009395,  39.92449853]),
 array([127.79612,  53.0025 ]),
 array([0.97710344, 0.33774151]))

In [15]:
pairs = []
for i in range(len(data)-1):
    for j in range(i+1, len(data)):
        pairs.append([i,j])
pairs = np.array(pairs)
print(len(pairs))

93528


In [16]:
#I = np.arange(len(pairs))
#np.random.shuffle(I)
#pairs_100 = pairs[I[:100]]
# we did above and chose the following pairs
pairs_100 = np.array([[ 87, 166], [161, 396], [370, 425], [209, 421], [ 84, 278], 
                      [ 24,  95], [ 72, 249], [358, 419], [ 86, 382], [241, 297],
                      [315, 363], [273, 335], [118, 270], [323, 430], [ 30, 431],
                      [111, 297], [ 17, 103], [191, 210], [179, 240], [ 53, 139],
                      [156, 240], [128, 412], [  8, 166], [ 96,  98], [270, 303],
                      [160, 319], [ 17,  41], [ 46, 381], [165, 216], [  8, 148],
                      [231, 287], [293, 426], [131, 264], [136, 271], [ 69, 137],
                      [120, 139], [111, 261], [ 13, 114], [138, 291], [ 31, 414], 
                      [252, 276], [ 38, 126], [ 31, 243], [382, 417], [240, 262],
                      [ 45,  49], [313, 359], [107, 206], [212, 243], [ 91, 383],
                      [118, 402], [ 31, 355], [ 74, 365], [110, 132], [136, 330],
                      [326, 376], [ 28, 128], [258, 302], [193, 256], [ 48, 151],
                      [255, 309], [340, 391], [ 90, 216], [341, 400], [ 45, 296], 
                      [206, 370], [184, 379], [248, 343], [ 16, 271], [245, 401],
                      [ 25, 130], [ 15, 430], [ 20, 285], [362, 385], [192, 343], 
                      [111, 410], [400, 414], [116, 185], [ 48, 202], [ 85, 234], 
                      [281, 392], [318, 320], [ 85, 417], [390, 427], [357, 428], 
                      [150, 189], [ 40, 343], [173, 249], [ 38, 399], [121, 393], 
                      [152, 316], [  0, 141], [ 24, 397], [255, 424], [ 92, 330], 
                      [121, 245], [ 78, 240], [ 68, 255], [ 21, 288], [278, 399]])

In [17]:
print(pairs_100.tolist())

[[87, 166], [161, 396], [370, 425], [209, 421], [84, 278], [24, 95], [72, 249], [358, 419], [86, 382], [241, 297], [315, 363], [273, 335], [118, 270], [323, 430], [30, 431], [111, 297], [17, 103], [191, 210], [179, 240], [53, 139], [156, 240], [128, 412], [8, 166], [96, 98], [270, 303], [160, 319], [17, 41], [46, 381], [165, 216], [8, 148], [231, 287], [293, 426], [131, 264], [136, 271], [69, 137], [120, 139], [111, 261], [13, 114], [138, 291], [31, 414], [252, 276], [38, 126], [31, 243], [382, 417], [240, 262], [45, 49], [313, 359], [107, 206], [212, 243], [91, 383], [118, 402], [31, 355], [74, 365], [110, 132], [136, 330], [326, 376], [28, 128], [258, 302], [193, 256], [48, 151], [255, 309], [340, 391], [90, 216], [341, 400], [45, 296], [206, 370], [184, 379], [248, 343], [16, 271], [245, 401], [25, 130], [15, 430], [20, 285], [362, 385], [192, 343], [111, 410], [400, 414], [116, 185], [48, 202], [85, 234], [281, 392], [318, 320], [85, 417], [390, 427], [357, 428], [150, 189], [40, 3

In [18]:
taxi_ids_selected = list(set(pairs_100.reshape(200,)))
print("Chosen Users: \n", taxi_ids_selected)

Chosen Users: 
 [0, 8, 13, 15, 16, 17, 20, 21, 24, 25, 28, 30, 31, 38, 40, 41, 45, 46, 48, 49, 53, 68, 69, 72, 74, 78, 84, 85, 86, 87, 90, 91, 92, 95, 96, 98, 103, 107, 110, 111, 114, 116, 118, 120, 121, 126, 128, 130, 131, 132, 136, 137, 138, 139, 141, 148, 150, 151, 152, 156, 160, 161, 165, 166, 173, 179, 184, 185, 189, 191, 192, 193, 202, 206, 209, 210, 212, 216, 231, 234, 240, 241, 243, 245, 248, 249, 252, 255, 256, 258, 261, 262, 264, 270, 271, 273, 276, 278, 281, 285, 287, 288, 291, 293, 296, 297, 302, 303, 309, 313, 315, 316, 318, 319, 320, 323, 326, 330, 335, 340, 341, 343, 355, 357, 358, 359, 362, 363, 365, 370, 376, 379, 381, 382, 383, 385, 390, 391, 392, 393, 396, 397, 399, 400, 401, 402, 410, 412, 414, 417, 419, 421, 424, 425, 426, 427, 428, 430, 431]


In [19]:
print("Chosen Users From Data:")
taxi_idxs[taxi_ids_selected] + 1

Chosen Users From Data:


array([10, 251, 817, 820, 839, 841, 875, 877, 919, 947, 1112, 1327, 1350,
       1378, 1439, 1466, 1557, 1563, 1567, 1570, 1725, 1825, 1830, 1847,
       1909, 1950, 2127, 2128, 2138, 2146, 2157, 2193, 2205, 2240, 2274,
       2313, 2454, 2550, 2566, 2567, 2594, 2617, 2862, 2876, 2880, 2909,
       2924, 2927, 2938, 2943, 3142, 3154, 3259, 3260, 3511, 3772, 3866,
       3872, 3896, 3939, 3953, 3954, 3965, 3968, 4312, 4393, 4415, 4452,
       4527, 4580, 4582, 4590, 4730, 4830, 4854, 4875, 4889, 4969, 5415,
       5605, 5896, 5921, 5970, 6168, 6274, 6286, 6361, 6427, 6464, 6569,
       6611, 6623, 6652, 6790, 6834, 6876, 6899, 6908, 6970, 6997, 7008,
       7016, 7133, 7139, 7193, 7199, 7347, 7376, 7411, 7434, 7467, 7469,
       7624, 7653, 7674, 7790, 7840, 7971, 8063, 8260, 8275, 8367, 8654,
       8672, 8676, 8696, 8731, 8736, 8764, 8833, 9186, 9241, 9275, 9278,
       9290, 9316, 9354, 9356, 9368, 9395, 9438, 9468, 9481, 9500, 9513,
       9525, 9788, 9812, 9837, 9946, 10012, 10082,

In [20]:
data = data[taxi_ids_selected]
data.shape

(159,)

In [21]:
Dict = {taxi_ids_selected[i]:i for i in range(len(taxi_ids_selected))} 
pairs_new = []
for pair in pairs_100:
    pairs_new.append([Dict[pair[0]], Dict[pair[1]]])
pairs_new

[[29, 63],
 [61, 140],
 [129, 153],
 [74, 151],
 [26, 97],
 [8, 33],
 [23, 85],
 [124, 150],
 [28, 133],
 [81, 105],
 [110, 127],
 [95, 118],
 [42, 93],
 [115, 157],
 [11, 158],
 [39, 105],
 [5, 36],
 [69, 75],
 [65, 80],
 [20, 53],
 [59, 80],
 [46, 147],
 [1, 63],
 [34, 35],
 [93, 107],
 [60, 113],
 [5, 15],
 [17, 132],
 [62, 77],
 [1, 55],
 [78, 100],
 [103, 154],
 [48, 92],
 [50, 94],
 [22, 51],
 [43, 53],
 [39, 90],
 [2, 40],
 [52, 102],
 [12, 148],
 [86, 96],
 [13, 45],
 [12, 82],
 [133, 149],
 [80, 91],
 [16, 19],
 [109, 125],
 [37, 73],
 [76, 82],
 [31, 134],
 [42, 145],
 [12, 122],
 [24, 128],
 [38, 49],
 [50, 117],
 [116, 130],
 [10, 46],
 [89, 106],
 [71, 88],
 [18, 57],
 [87, 108],
 [119, 137],
 [30, 77],
 [120, 143],
 [16, 104],
 [73, 129],
 [66, 131],
 [84, 121],
 [4, 94],
 [83, 144],
 [9, 47],
 [3, 157],
 [6, 99],
 [126, 135],
 [70, 121],
 [39, 146],
 [143, 148],
 [41, 67],
 [18, 72],
 [27, 79],
 [98, 138],
 [112, 114],
 [27, 149],
 [136, 155],
 [123, 156],
 [56, 68],
 [1

In [22]:
data_with_time = data + 0

In [23]:
for i in range(len(data)):
    for j in range(len(data[i])):
        data[i][j] = data[i][j][:,1:]

In [24]:
Min = np.min([np.min([np.min(data[i][j], 0) for j in range(len(data[i]))], 0) for i in range(len(data))], 0)
Max = np.max([np.max([np.max(data[i][j], 0) for j in range(len(data[i]))], 0) for i in range(len(data))], 0)
Mean = np.mean([np.mean([np.mean(data[i][j], 0) for j in range(len(data[i]))], 0) for i in range(len(data))], 0)
Std = np.std([np.std([np.std(data[i][j], 0) for j in range(len(data[i]))], 0) for i in range(len(data))], 0)
Min, Mean, Max, Std, LA.norm(Max - Mean)

(array([0., 0.]),
 array([116.40432008,  39.92667049]),
 array([127.79612,  53.0025 ]),
 array([0.88903461, 0.30846359]),
 17.34215738681324)

In [25]:
interesting_idx = []
for i in range(len(data)):
    I = np.where(np.array([np.min(data[i][j][:,0]) for j in range(len(data[i]))]) < 20)[0]
    if len(I) > 0:
        interesting_idx.append(i)
        
print(interesting_idx)

[10, 21, 28, 35, 76, 84, 85, 86, 93, 96, 97, 102, 108, 115, 117, 124, 149]


In [26]:
print("Final Chosen Users From Data:")
taxi_idxs[taxi_ids_selected][interesting_idx] + 1

Final Chosen Users From Data:


array([1112, 1825, 2138, 2313, 4889, 6274, 6286, 6361, 6790, 6899, 6908,
       7133, 7411, 7790, 7971, 8676, 9946], dtype=object)

In [27]:
for i in interesting_idx:
    print(i, set([len(data[i][j]) for j in range(len(data[i]))]))

10 {10, 11, 12, 13, 18}
21 {10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 54, 55, 56, 58, 59, 60, 61, 63, 66, 67, 75, 77, 81, 82}
28 {12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 59, 61, 62, 65, 66, 76}
35 {11, 14, 15, 16, 18, 19, 20, 23, 24, 26, 28, 29, 32, 33, 34, 37, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 73, 74, 75, 76, 77, 79, 80, 81, 82, 83, 84, 87, 91, 92, 95, 96, 100, 101, 102, 120, 126}
76 {10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 54, 58, 59, 60, 61, 70}
84 {10, 11, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,

# Classifiers

In [168]:
CC = [100, 100, 1000]
number_estimators = 50

clf1 = [make_pipeline(LinearSVC(dual=False, C=CC[0], tol=1e-5, 
                               class_weight ='balanced', max_iter=1000)), 
        "SVM, LinearSVC, C = "+str(CC[0])]
clf2 = [make_pipeline(StandardScaler(), svm.SVC(C=CC[1], kernel='rbf', gamma='auto', max_iter=200000)),
        "Gaussian SVM, C="+str(CC[1])+", gamma=auto"]
clf3 = [make_pipeline(StandardScaler(), svm.SVC(C=CC[2], kernel='poly', degree=3, max_iter=400000)),
        "Poly kernel SVM, C="+str(CC[2])+", deg=auto"]
clf4 = [DecisionTreeClassifier(), "Decision Tree"]
clf5 = [RandomForestClassifier(n_estimators=number_estimators), 
         "RandomForestClassifier, n="+str(number_estimators)]
clf6 = [KNeighborsClassifier(n_neighbors=5), "KNN"]
clf7 = [LogisticRegression(solver='lbfgs'), "Logistic Regression"]

clf = [clf1, clf2, clf3, clf4, clf5, clf6, clf7]

classifs = [item[0] for item in clf]
keys = [item[1] for item in clf]

In [33]:
def classification_without_exp_random_Q(data_1, data_2, Q_size, epoch, classifiers):
    start_time = time.time()
    models = [item[0] for item in classifiers]
    keys = [item[1] for item in classifiers]

    r = len(classifiers)

    train_error_mean = np.zeros(r)
    test_error_mean = np.zeros(r)
    test_error_std = np.zeros(r)
    
    train_errors = np.zeros((r, epoch)) 
    test_errors = np.zeros((r, epoch))

    n_1 = len(data_1)
    n_2 = len(data_2) 

    for s in range(epoch):
        I1, I2, I3, I4 = train_test_split(np.arange(n_1), [1] * n_1, test_size=0.3) 
        J1, J2, J3, J4 = train_test_split(np.arange(n_2), [-1] * n_2, test_size=0.3)

        x_preds = np.zeros((r, len(I1) + len(J1)))
        y_preds = np.zeros((r, len(I2) + len(J2)))

        train = np.concatenate((data_1[I1], data_2[J1]), 0)
        test = np.concatenate((data_1[I2], data_2[J2]), 0)

        Min = np.min([np.min(train[i], 0) for i in range(len(train))], 0)
        Max = np.max([np.max(train[i], 0) for i in range(len(train))], 0)
        Mean = np.mean([np.mean(train[i], 0) for i in range(len(train))], 0)
        Std = np.std([np.std(train[i], 0) for i in range(len(train))], 0)
       
        Q = np.ones((Q_size, 2))
        Q[:,0] = np.random.normal(Mean[0], 4 * Std[0], Q_size)
        Q[:,1] = np.random.normal(Mean[1], 4 * Std[1], Q_size)

        X = curve2vec(Q, train)
        y = np.concatenate((I3, J3), axis = 0)

        test_data = curve2vec(Q, test)
        test_labels = np.concatenate((I4, J4), axis = 0)

        for k in range(r): 
            model = models[k]
            model.fit(X, y)

            x_preds[k] = model.predict(X)                
            y_preds[k] = model.predict(test_data)
        
        for k in range(r):
            train_errors[k][s] = 1 - metrics.accuracy_score(y, x_preds[k])
            test_errors[k][s] = 1 - metrics.accuracy_score(test_labels, y_preds[k])
            
    for k in range(r):
        train_error_mean[k] = np.mean(train_errors[k])
        test_error_mean[k] = np.mean(test_errors[k])
        test_error_std[k] = np.std(test_errors[k])
    
    Dict = {}

    for k in range(len(keys)): 
        Dict[k+1] = [keys[k], np.round(train_error_mean[k], decimals = 4), 
                     np.round(test_error_mean[k], decimals = 4),
                     np.round(test_error_std[k], decimals = 4)]

    pdf = pd.DataFrame.from_dict(Dict, orient='index', 
                columns=['Classifier','Train Error', 'Test Error', 'Std Error'])
    
    print(colored(f"total time = {time.time() - start_time}", "red"))

    return pdf, train_error_mean, test_error_mean

In [29]:
pairs_100

array([[ 87, 166],
       [161, 396],
       [370, 425],
       [209, 421],
       [ 84, 278],
       [ 24,  95],
       [ 72, 249],
       [358, 419],
       [ 86, 382],
       [241, 297],
       [315, 363],
       [273, 335],
       [118, 270],
       [323, 430],
       [ 30, 431],
       [111, 297],
       [ 17, 103],
       [191, 210],
       [179, 240],
       [ 53, 139],
       [156, 240],
       [128, 412],
       [  8, 166],
       [ 96,  98],
       [270, 303],
       [160, 319],
       [ 17,  41],
       [ 46, 381],
       [165, 216],
       [  8, 148],
       [231, 287],
       [293, 426],
       [131, 264],
       [136, 271],
       [ 69, 137],
       [120, 139],
       [111, 261],
       [ 13, 114],
       [138, 291],
       [ 31, 414],
       [252, 276],
       [ 38, 126],
       [ 31, 243],
       [382, 417],
       [240, 262],
       [ 45,  49],
       [313, 359],
       [107, 206],
       [212, 243],
       [ 91, 383],
       [118, 402],
       [ 31, 355],
       [ 74,

In [34]:
taxi_idxs[taxi_ids_selected]

array([9, 250, 816, 819, 838, 840, 874, 876, 918, 946, 1111, 1326, 1349,
       1377, 1438, 1465, 1556, 1562, 1566, 1569, 1724, 1824, 1829, 1846,
       1908, 1949, 2126, 2127, 2137, 2145, 2156, 2192, 2204, 2239, 2273,
       2312, 2453, 2549, 2565, 2566, 2593, 2616, 2861, 2875, 2879, 2908,
       2923, 2926, 2937, 2942, 3141, 3153, 3258, 3259, 3510, 3771, 3865,
       3871, 3895, 3938, 3952, 3953, 3964, 3967, 4311, 4392, 4414, 4451,
       4526, 4579, 4581, 4589, 4729, 4829, 4853, 4874, 4888, 4968, 5414,
       5604, 5895, 5920, 5969, 6167, 6273, 6285, 6360, 6426, 6463, 6568,
       6610, 6622, 6651, 6789, 6833, 6875, 6898, 6907, 6969, 6996, 7007,
       7015, 7132, 7138, 7192, 7198, 7346, 7375, 7410, 7433, 7466, 7468,
       7623, 7652, 7673, 7789, 7839, 7970, 8062, 8259, 8274, 8366, 8653,
       8671, 8675, 8695, 8730, 8735, 8763, 8832, 9185, 9240, 9274, 9277,
       9289, 9315, 9353, 9355, 9367, 9394, 9437, 9467, 9480, 9499, 9512,
       9524, 9787, 9811, 9836, 9945, 10011, 10081, 

In [27]:
pairs_errors = []
i = 0
for pair in pairs_new:
    A = classification_without_exp_random_Q(data[pair[0]], data[pair[1]], Q_size=20, 
                                            epoch=100, classifiers=[clf12])
    pairs_errors.append([pair, np.round(A[-1][0], decimals=2)])
    print('i =', i)
    print(colored(f"pair={pair}", 'green'))
    print(A[0])
    print("===================================================================")
    i += 1

total time = 9.3976309299469
i = 0
pair=[29, 63]
                     Classifier  Train Error  Test Error  Std Error
1  RandomForestClassifier, n=50       0.0005      0.1795     0.0378
total time = 10.600561141967773
i = 1
pair=[61, 140]
                     Classifier  Train Error  Test Error  Std Error
1  RandomForestClassifier, n=50       0.0005      0.2851     0.0421
total time = 10.537057161331177
i = 2
pair=[129, 153]
                     Classifier  Train Error  Test Error  Std Error
1  RandomForestClassifier, n=50       0.0005      0.2505     0.0467
total time = 11.264155864715576
i = 3
pair=[74, 151]
                     Classifier  Train Error  Test Error  Std Error
1  RandomForestClassifier, n=50       0.0002      0.2038     0.0354
total time = 12.360167264938354
i = 4
pair=[26, 97]
                     Classifier  Train Error  Test Error  Std Error
1  RandomForestClassifier, n=50       0.0008      0.1374     0.0286
total time = 12.210195064544678
i = 5
pair=[8, 33]
        

In [32]:
print(f"selected 1oo random pairs of taxies in T-Drive dataset: \n {pairs_100.tolist()}")
print(f"selected random 100 pairs of taxies in data dataset: \n {pairs_new}")
#selected_pairs_of_taxies = 

selected 1oo random pairs of taxies in T-Drive dataset: 
 [[87, 166], [161, 396], [370, 425], [209, 421], [84, 278], [24, 95], [72, 249], [358, 419], [86, 382], [241, 297], [315, 363], [273, 335], [118, 270], [323, 430], [30, 431], [111, 297], [17, 103], [191, 210], [179, 240], [53, 139], [156, 240], [128, 412], [8, 166], [96, 98], [270, 303], [160, 319], [17, 41], [46, 381], [165, 216], [8, 148], [231, 287], [293, 426], [131, 264], [136, 271], [69, 137], [120, 139], [111, 261], [13, 114], [138, 291], [31, 414], [252, 276], [38, 126], [31, 243], [382, 417], [240, 262], [45, 49], [313, 359], [107, 206], [212, 243], [91, 383], [118, 402], [31, 355], [74, 365], [110, 132], [136, 330], [326, 376], [28, 128], [258, 302], [193, 256], [48, 151], [255, 309], [340, 391], [90, 216], [341, 400], [45, 296], [206, 370], [184, 379], [248, 343], [16, 271], [245, 401], [25, 130], [15, 430], [20, 285], [362, 385], [192, 343], [111, 410], [400, 414], [116, 185], [48, 202], [85, 234], [281, 392], [318, 3

In [35]:
#print(f"test errors with Random Forest: \n {list(np.array(pairs_errors, dtype='object')[:,1])}")
test_errors_RF = [0.18, 0.29, 0.25, 0.2, 0.14, 0.07, 0.11, 0.04, 0.19, 0.24, 0.24, 0.26, 
                  0.23, 0.2, 0.17, 0.25, 0.07, 0.23, 0.18, 0.11, 0.32, 0.19, 0.29, 0.17, 
                  0.16, 0.23, 0.11, 0.18, 0.17, 0.36, 0.17, 0.2, 0.33, 0.37, 0.26, 0.21, 
                  0.24, 0.35, 0.14, 0.26, 0.14, 0.24, 0.21, 0.16, 0.22, 0.26, 0.2, 0.24, 
                  0.1, 0.21, 0.31, 0.13, 0.11, 0.33, 0.07, 0.12, 0.2, 0.32, 0.14, 0.17, 
                  0.19, 0.27, 0.27, 0.06, 0.23, 0.14, 0.22, 0.24, 0.16, 0.3, 0.18, 0.28, 
                  0.15, 0.11, 0.28, 0.26, 0.07, 0.31, 0.18, 0.2, 0.26, 0.23, 0.25, 0.3, 
                  0.13, 0.27, 0.25, 0.16, 0.14, 0.08, 0.19, 0.11, 0.22, 0.19, 0.07, 0.19, 
                  0.33, 0.19, 0.23, 0.24]
print(test_errors_RF)

[0.18, 0.29, 0.25, 0.2, 0.14, 0.07, 0.11, 0.04, 0.19, 0.24, 0.24, 0.26, 0.23, 0.2, 0.17, 0.25, 0.07, 0.23, 0.18, 0.11, 0.32, 0.19, 0.29, 0.17, 0.16, 0.23, 0.11, 0.18, 0.17, 0.36, 0.17, 0.2, 0.33, 0.37, 0.26, 0.21, 0.24, 0.35, 0.14, 0.26, 0.14, 0.24, 0.21, 0.16, 0.22, 0.26, 0.2, 0.24, 0.1, 0.21, 0.31, 0.13, 0.11, 0.33, 0.07, 0.12, 0.2, 0.32, 0.14, 0.17, 0.19, 0.27, 0.27, 0.06, 0.23, 0.14, 0.22, 0.24, 0.16, 0.3, 0.18, 0.28, 0.15, 0.11, 0.28, 0.26, 0.07, 0.31, 0.18, 0.2, 0.26, 0.23, 0.25, 0.3, 0.13, 0.27, 0.25, 0.16, 0.14, 0.08, 0.19, 0.11, 0.22, 0.19, 0.07, 0.19, 0.33, 0.19, 0.23, 0.24]


### The following two boxes are for above 0.20 test error that newly are done

In [36]:
pairs_100_ = pairs_100 + 0

In [37]:
#pairs_20_ = pairs_100[np.where(np.array(pairs_errors, dtype='object')[:,1] > 0.20)[0]]
#pairs_20 = np.array(pairs_new)[np.where(np.array(pairs_errors, dtype='object')[:,1] > 0.20)[0]]

pairs_20_ = [[61, 140], [129, 153], [81, 105], [110, 127], [95, 118], [42, 93], [39, 105],
             [69, 75], [59, 80], [1, 63], [60, 113], [1, 55], [48, 92], [50, 94], [22, 51],
             [43, 53], [39, 90], [2, 40], [12, 148], [13, 45], [12, 82],  [80, 91], 
             [16, 19], [37, 73], [31, 134], [42, 145], [38, 49], [89, 106], [119, 137], 
             [30, 77], [16, 104], [66, 131], [84, 121], [83, 144], [3, 157], [70, 121], 
             [39, 146], [41, 67], [98, 138], [112, 114], [27, 149], [136, 155], [56, 68], 
             [14, 121], [8, 141], [25, 80], [7, 101], [97, 142]]

pairs_20 = [[ 61, 140], [129, 153], [ 81, 105], [110, 127], [ 95, 118], [ 42,  93], 
            [ 39, 105], [ 69,  75], [ 59,  80], [  1,  63], [ 60, 113], [  1,  55], 
            [ 48,  92], [ 50,  94], [ 22,  51], [ 43,  53], [ 39,  90], [  2,  40], 
            [ 12, 148], [ 13,  45], [ 12,  82], [ 80,  91], [ 16,  19], [ 37,  73], 
            [ 31, 134], [ 42, 145], [ 38,  49], [ 89, 106], [119, 137], [ 30,  77], 
            [ 16, 104], [ 66, 131], [ 84, 121], [ 83, 144], [  3, 157], [ 70, 121], 
            [ 39, 146], [ 41,  67], [ 98, 138], [112, 114], [ 27, 149], [136, 155], 
            [ 56,  68], [ 14, 121], [  8, 141], [ 25,  80], [  7, 101], [ 97, 142]]


print(len(pairs_20))
print(pairs_20_)
print(pairs_20)

48
[[61, 140], [129, 153], [81, 105], [110, 127], [95, 118], [42, 93], [39, 105], [69, 75], [59, 80], [1, 63], [60, 113], [1, 55], [48, 92], [50, 94], [22, 51], [43, 53], [39, 90], [2, 40], [12, 148], [13, 45], [12, 82], [80, 91], [16, 19], [37, 73], [31, 134], [42, 145], [38, 49], [89, 106], [119, 137], [30, 77], [16, 104], [66, 131], [84, 121], [83, 144], [3, 157], [70, 121], [39, 146], [41, 67], [98, 138], [112, 114], [27, 149], [136, 155], [56, 68], [14, 121], [8, 141], [25, 80], [7, 101], [97, 142]]
[[61, 140], [129, 153], [81, 105], [110, 127], [95, 118], [42, 93], [39, 105], [69, 75], [59, 80], [1, 63], [60, 113], [1, 55], [48, 92], [50, 94], [22, 51], [43, 53], [39, 90], [2, 40], [12, 148], [13, 45], [12, 82], [80, 91], [16, 19], [37, 73], [31, 134], [42, 145], [38, 49], [89, 106], [119, 137], [30, 77], [16, 104], [66, 131], [84, 121], [83, 144], [3, 157], [70, 121], [39, 146], [41, 67], [98, 138], [112, 114], [27, 149], [136, 155], [56, 68], [14, 121], [8, 141], [25, 80], [7, 

In [74]:
pairs_20_string = ['[ 61 140]', '[129 153]', '[ 81 105]', '[110 127]', '[ 95 118]', 
                   '[ 42  93]', '[ 39 105]', '[ 69  75]', '[ 59  80]', '[  1  63]', 
                   '[ 60 113]', '[  1  55]', '[ 48  92]', '[ 50  94]', '[ 22  51]', 
                   '[ 43  53]', '[ 39  90]', '[  2  40]', '[ 12 148]', '[ 13  45]', 
                   '[ 12  82]', '[ 80  91]', '[ 16  19]', '[ 37  73]', '[ 31 134]', 
                   '[ 42 145]', '[ 38  49]', '[ 89 106]', '[119 137]', '[ 30  77]', 
                   '[ 16 104]', '[ 66 131]', '[ 84 121]', '[ 83 144]', '[  3 157]', 
                   '[ 70 121]', '[ 39 146]', '[ 41  67]', '[ 98 138]', '[112 114]',
                   '[ 27 149]', '[136 155]', '[ 56  68]', '[ 14 121]', '[  8 141]', 
                   '[ 25  80]', '[  7 101]', '[ 97 142]']

In [38]:
pairs_100 = np.array(pairs_new, dtype='object') + 0

In [ ]:
#TP = pairs_in_users_in_data
pairs_in_users_in_data = [[T[pairs_20[i][0]], T[pairs_20[i][1]]] for i in range(len(pairs_20))]
print("Chosen pairs from data: \n", pairs_in_users_in_data)
pairs_in_users_in_data = np.array(pairs_in_users_in_data)
print(len(pairs_in_users_in_data))

#pairs_in_users_in_data = [[3953, 9437], [8832, 10168], [5920, 7198], [7466, 8735], 
#[6875, 8062], [2861, 6789], [2566, 7198], [4579, 4874], [3938, 5895], [250, 3967], 
#[3952, 7652], [250, 3771], [2937, 6651], [3141, 6833], [1829, 3153], [2875, 3259], 
#[2566, 6610], [816, 2593], [1349, 9836], [1377, 2908], [1349, 5969], [5895, 6622], 
#[1556, 1569], [2549, 4829], [2192, 9289], [2861, 9524], [2565, 2942], [6568, 7346], 
#[8259, 9355], [2156, 4968], [1556, 7192], [4414, 9240], [6273, 8366], [6167, 9512], 
#[819, 10200], [4581, 8366], [2566, 9787], [2616, 4451], [6969, 9367], [7623, 7673], 
#[2127, 9945], [9353, 10179], [3865, 4526], [1438, 8366], [918, 9467], [1949, 5895], 
#[876, 7015], [6907, 9480]]

# Generating distance matrices

## $d_Q^{\pi}$

In [85]:
def calculate_dists_d_Q_pi(data1, data2, p, path): 
    start_time = time.time() 
    data = np.concatenate((data1, data2), 0)
    n = len(data)
    A = []
    for i in range(n-1):
        for j in range(i+1, n):
            A.append(d_Q_pi(Q, data[i], data[j], p=p))
    A = np.array(A)
    tri = np.zeros((n, n))
    tri[np.triu_indices(n, 1)] = A
    for i in range(1, n):
        for j in range(i):
            tri[i][j] = tri[j][i]
    np.savetxt(path, tri, delimiter=',')

    total_time = time.time() - start_time
    return total_time

In [ ]:
# the following path is for 100 chosen pairs for correlation and R^2 test
paths = []
for i in range(len(pairs_100)): 
    paths.append('Calculated Distance Matrices for KNN for comparison with RF/Taxi/d_Q_pi/Taxi-Pairs['+str(pairs_100[i])+']-d_Q_pi.csv')

for i in range(len(pairs_100)):
    calculate_dists_d_Q_pi(data[pairs_100[i][0]], data[pairs_100[i][1]], p=1, path=paths[i])


In [ ]:
# the following path is for above 0.20 test errors
paths = []
for i in range(len(pairs_20)):
    paths.append('Calculated Distance Matrices for KNN Above 0.20/d_Q_pi/Taxi-Pairs'+str(pairs_20[i])+'-d_Q_pi.csv')

for i in range(len(pairs_20)):
    calculate_dists_d_Q_pi(data[pairs_20[i][0]], data[pairs_20[i][1]], p=1, path=paths[i])


In [40]:
def KNN_with_d_Q_pi(n_1, n_2, dist_matrix):
    
    '''path example: '/Users/hasan/Desktop/Anaconda/Research/Calculated Distance Matrices for KNN/Taxi-Pairs['+str(pairs_final[i])+']-d_Q_pi.csv'
       path_to_dists: the path to the corresponding distance matrix
       n_1: len(data_1)
       n_2: len(data_2)'''

    I_1, J_1, y_train_1, y_test_1 = train_test_split(np.arange(n_1), 
                                                np.ones(n_1), test_size=0.3)
    I_2, J_2, y_train_2, y_test_2 = train_test_split(np.arange(n_1, n_1+n_2), 
                                                np.ones(n_2), test_size=0.3)
    labels = np.array([1] * n_1 + [0] * n_2)
    I = np.concatenate((I_1, I_2), 0)
    np.random.shuffle(I)
    J = np.concatenate((J_1, J_2), 0)
    np.random.shuffle(J)

    D_train = dist_matrix[I][:, I]
    D_test = dist_matrix[J][:,I]
    train_labels = labels[I]
    test_labels = labels[J]

    clf = KNeighborsClassifier(n_neighbors=5, metric='precomputed')
    
    #Train the model using the training sets
    clf.fit(D_train, list(train_labels))

    #Predict labels for train dataset
    train_pred = clf.predict(D_train)
    train_error = sum(train_labels != train_pred)/len(I)
    
    #Predict labels for test dataset
    test_pred = clf.predict(D_test)
    test_error = sum((test_labels != test_pred))/len(J)
        
    return train_error, test_error

In [41]:
def KNN_average_error_d_Q_pi(data1, data2, num_trials, path_to_dists, pair):

    '''path_to_dists: the path to the corresponding distance matrix'''

    Start_time = time.time()

    train_errors = np.zeros(num_trials)
    test_errors = np.zeros(num_trials)
    
    dist_matrix = np.array(pd.read_csv(path_to_dists, header=None))

    for i in range(num_trials):
        train_errors[i], test_errors[i] = KNN_with_d_Q_pi(len(data1), len(data2), dist_matrix)

    Dict = {}
    Dict[1] = [f"KNN with d_Q_pi for pairs {pair}", 
                    np.round(np.mean(train_errors), decimals = 4), 
                    np.round(np.mean(test_errors), decimals = 4), 
                    np.round(np.std(test_errors), decimals = 4)]

    df = pd.DataFrame.from_dict(Dict, orient='index', columns=['Classifier',
                                'Train Error', 'Test Error', 'std'])
    print(colored(f"num_trials = {num_trials}", "blue"))
    print(colored(f'total time = {time.time() - Start_time}', 'green'))

    return (df, np.mean(train_errors), np.mean(test_errors), np.median(test_errors), 
            np.std(test_errors))

In [ ]:
## the following path is for 100 chosen pairs for correlation and R^2 test
paths = []
for i in range(len(pairs_100)):
    paths.append('/Users/hasan/Desktop/Anaconda/Research/Calculated Distance Matrices for KNN for comparison with RF/Taxi/d_Q_pi/Taxi-Pairs['+str(pairs_100[i])+']-d_Q_pi.csv')

train_test_median_std_errors_d_Q_pi = []
for i in range(len(pairs_100)):
    print("i=", i)
    print(colored(f"pair: {pairs_100[i]}", 'magenta'))
    A = KNN_average_error_d_Q_pi(data[pairs_100[i][0]], data[pairs_100[i][1]], 
                                 num_trials=100, path_to_dists=paths[i], pair=pairs_100[i])
    train_test_median_std_errors_d_Q_pi.append(A[1:])
    print(A[0])
    print("=======================================================================")

In [679]:
path = '/Users/hasan/Desktop/Anaconda/Research/Calculated KNN errors for comparison with RF/Taxi-Pairs-d_Q_pi.csv'
np.savetxt(path, np.array(train_test_median_std_errors_d_Q_pi), delimiter=',')


In [60]:
# the following path is for above 0.20 test errors
train_test_median_std_errors_d_Q_pi_20 = []
paths = []
for i in range(len(pairs_20)):
    paths.append('Calculated Distance Matrices for KNN Above 0.20/d_Q_pi/Taxi-Pairs['+str(pairs_20[i])+']-d_Q_pi.csv')

for i in range(len(pairs_20)):
    print(colored(f"pair: {pairs_20[i]}", 'magenta'))
    A = KNN_average_error_d_Q_pi(data[pairs_20[i][0]], data[pairs_20[i][1]], 
                                   num_trials=1000, path_to_dists=paths[i], 
                                   pair=pairs_20[i])
    train_test_median_std_errors_d_Q_pi_20.append(A[1:])
    print(A[0])
    print("====================================================================")

pair: [ 61 140]
num_trials = 1000
total time = 8.531124353408813
                            Classifier  Train Error  Test Error     std
1  KNN with d_Q_pi for pairs [ 61 140]       0.2072      0.3006  0.0434
pair: [129 153]
num_trials = 1000
total time = 7.436275959014893
                            Classifier  Train Error  Test Error     std
1  KNN with d_Q_pi for pairs [129 153]       0.1999      0.2859  0.0442
pair: [ 81 105]
num_trials = 1000
total time = 9.986772060394287
                            Classifier  Train Error  Test Error    std
1  KNN with d_Q_pi for pairs [ 81 105]       0.1806      0.2507  0.037
pair: [110 127]
num_trials = 1000
total time = 7.6639509201049805
                            Classifier  Train Error  Test Error     std
1  KNN with d_Q_pi for pairs [110 127]       0.1494      0.2247  0.0423
pair: [ 95 118]
num_trials = 1000
total time = 9.507699012756348
                            Classifier  Train Error  Test Error     std
1  KNN with d_Q_pi for pairs

In [61]:
path = 'Calculated KNN errors for comparison with RF 20/Taxi-Pairs-d_Q_pi.csv'
np.savetxt(path, np.array(train_test_median_std_errors_d_Q_pi_20), delimiter=',')

## dtw from tslearn

In [42]:
def calculate_dists_dtw_tslearn(data1, data2, path): 
    start_time = time.time() 
    data = np.concatenate((data1, data2), 0)
    n = len(data)
    A = []
    for i in range(n-1):
        for j in range(i+1, n):
            A.append(tslearn.metrics.dtw(data[i], data[j]))
    A = np.array(A)
    tri = np.zeros((n, n))
    tri[np.triu_indices(n, 1)] = A
    for i in range(1, n):
        for j in range(i):
            tri[i][j] = tri[j][i]
    np.savetxt(path, tri, delimiter=',')

    total_time = time.time() - start_time
    return total_time

In [180]:
# the following path is for 100 chosen pairs for correlation and R^2 test
paths = []
for i in range(len(pairs_100)): 
    paths.append('/Users/hasan/Desktop/Anaconda/Research/Calculated Distance Matrices for KNN for comparison with RF/dtw_tslearn/Taxi/Taxi-Pairs['+str(pairs_100[i])+']-dtw-tslearn.csv')

for i in range(len(pairs_100)):
    calculate_dists_dtw_tslearn(data[pairs_100[i][0]], data[pairs_100[i][1]], path=paths[i])
    

In [63]:
# the following path is for above 0.20 test errors
paths = []
for i in range(len(pairs_20)):
    paths.append('Calculated Distance Matrices for KNN Above 0.20/dtw-tslearn/Taxi-Pairs['+str(pairs_20[i])+']-dtw-tslearn.csv')

for i in range(len(pairs_20)):
    calculate_dists_dtw_tslearn(data[pairs_20[i][0]], data[pairs_20[i][1]], path=paths[i])


In [43]:
def KNN_with_dists_dtw_tslearn(n_1, n_2, path_to_dists):
    '''path example: '/Users/hasan/Desktop/Anaconda/Research/Calculated Distance Matrices for KNN/Taxi-Pairs['+str(pairs_final[i])+']-dtw-tslearn.csv'
       path_to_dists: the path to the corresponding distance matrix
       n_1: len(data_1)
       n_2: len(data_2)'''

    I_1, J_1, y_train_1, y_test_1 = train_test_split(np.arange(n_1), 
                                                np.ones(n_1), test_size=0.3)
    I_2, J_2, y_train_2, y_test_2 = train_test_split(np.arange(n_1, n_1+n_2), 
                                                np.ones(n_2), test_size=0.3)
    labels = np.array([1] * n_1 + [0] * n_2)
    I = np.concatenate((I_1, I_2), 0)
    np.random.shuffle(I)
    J = np.concatenate((J_1, J_2), 0)
    np.random.shuffle(J)
    dist_matrix = np.array(pd.read_csv(path_to_dists,  header=None))

    D_train = dist_matrix[I][:, I]
    D_test = dist_matrix[J][:,I]
    train_labels = labels[I]
    test_labels = labels[J]

    clf = KNeighborsClassifier(n_neighbors=5, metric='precomputed')
    
    #Train the model using the training sets
    clf.fit(D_train, list(train_labels))

    #Predict labels for train dataset
    train_pred = clf.predict(D_train)
    train_error = sum(train_labels != train_pred)/len(I)
    
    #Predict labels for test dataset
    test_pred = clf.predict(D_test)
    test_error = sum((test_labels != test_pred))/len(J)
        
    return train_error, test_error

In [44]:
def KNN_average_error_dtw_tslearn(data1, data2, num_trials, path_to_dists, pair):

    '''path_to_dists: the path to the corresponding distance matrix'''

    Start_time = time.time()

    train_errors = np.zeros(num_trials)
    test_errors = np.zeros(num_trials)

    for i in range(num_trials):
        train_errors[i], test_errors[i] = KNN_with_dists_dtw_tslearn(len(data1), len(data2), path_to_dists)

    Dict = {}
    Dict[1] = [f"KNN with dtw from tslearn for pairs {pair}", 
                    np.round(np.mean(train_errors), decimals = 4), 
                    np.round(np.mean(test_errors), decimals = 4), 
                    np.round(np.std(test_errors), decimals = 4)]

    df = pd.DataFrame.from_dict(Dict, orient='index', columns=['Classifier',
                                'Train Error', 'Test Error', 'std'])
    print(colored(f"num_trials = {num_trials}", "blue"))
    print(colored(f'total time = {time.time() - Start_time}', 'green'))

    return (df, np.mean(train_errors), np.mean(test_errors), np.median(test_errors), 
            np.std(test_errors))

In [ ]:
# # the following path is for 100 chosen pairs for correlation and R^2 test
paths = []
for i in range(len(pairs_100)):
    paths.append('/Users/hasan/Desktop/Anaconda/Research/Calculated Distance Matrices for KNN for comparison with RF/Taxi/dtw_tslearn/Taxi-Pairs['+str(pairs_100[i])+']-dtw-tslearn.csv')

train_test_median_std_errors_dtw_tslearn = []
for i in range(len(pairs_100)):
    print("i=", i)
    print(colored(f"pair: {pairs_100[i]}", 'magenta'))
    A = KNN_average_error_dtw_tslearn(data[pairs_100[i][0]], data[pairs_100[i][1]], num_trials=100, 
                            path_to_dists=paths[i], pair=pairs_100[i])
    train_test_median_std_errors_dtw_tslearn.append(A[1:])
    print(A[0])
    print("=====================================================================")

In [364]:
path = '/Users/hasan/Desktop/Anaconda/Research/Calculated KNN errors for comparison with RF/Taxi-Pairs-dtw_tslearn.csv'
np.savetxt(path, np.array(train_test_median_std_errors_dtw_tslearn), delimiter=',')


In [722]:
# test errors = np.array(train_test_median_std_errors_dtw_tslearn)[:,1]

In [66]:
# the following path is for above 0.20 test errors
paths = []
train_test_median_std_errors_dtw_tslearn_20 = []
for i in range(len(pairs_20)):
    paths.append('Calculated Distance Matrices for KNN Above 0.20/dtw-tslearn/Taxi-Pairs['+str(pairs_20[i])+']-dtw-tslearn.csv')

for i in range(len(pairs_20)):
    print(colored(f"i=, {i}, pair: {pairs_20[i]}", 'magenta'))
    A = KNN_average_error_dtw_tslearn(data[pairs_20[i][0]], data[pairs_20[i][1]], 
                                      num_trials=1000, path_to_dists=paths[i], 
                                      pair=pairs_20[i])
    train_test_median_std_errors_dtw_tslearn_20.append(A[1:])
    print(A[0])
    print("=============================================================================")

i=, 0, pair: [ 61 140]
num_trials = 1000
total time = 29.0521399974823
                                      Classifier  Train Error  Test Error  \
1  KNN with dtw from tslearn for pairs [ 61 140]       0.2043      0.2934   

      std  
1  0.0406  
i=, 1, pair: [129 153]
num_trials = 1000
total time = 24.393200874328613
                                      Classifier  Train Error  Test Error  \
1  KNN with dtw from tslearn for pairs [129 153]       0.2031      0.2834   

      std  
1  0.0453  
i=, 2, pair: [ 81 105]
num_trials = 1000
total time = 35.41420292854309
                                      Classifier  Train Error  Test Error  \
1  KNN with dtw from tslearn for pairs [ 81 105]       0.1852      0.2534   

      std  
1  0.0366  
i=, 3, pair: [110 127]
num_trials = 1000
total time = 27.22883892059326
                                      Classifier  Train Error  Test Error  \
1  KNN with dtw from tslearn for pairs [110 127]       0.1682      0.2404   

      std  
1  0.040

In [67]:
path = 'Calculated KNN errors for comparison with RF 20/Taxi-Pairs-dtw-tslearn.csv'
np.savetxt(path, np.array(train_test_median_std_errors_dtw_tslearn_20), delimiter=',')

## From github page: https://github.com/bguillouet/traj-dist

It includes 9 distances for trajectories including: Continuous Frechet, Discrete Frechet, Hausdorff, DTW, SSPD, LCSS, EDR, ERP.

## All but the continuous Frechet distance are really fast.

In [301]:
import traj_dist.distance as tdist
import pickle

def calculate_dists(data, metrics, pair, eps_edr, eps_lcss): 
    start_time = time.time() 
    Data = np.concatenate((data[pair[0]], data[pair[1]]), 0)
    n = len(Data)
    for metric in metrics:
        if metric == 'edr':
            A = tdist.pdist(Data, metric, eps=eps_edr)
        elif metric == 'lcss':
            A = tdist.pdist(Data, metric, eps=eps_lcss)
        else:
            A = tdist.pdist(Data, metric)
        tri = np.zeros((n, n))
        tri[np.triu_indices(n, 1)] = A
        for i in range(1, n):
            for j in range(i):
                tri[i][j] = tri[j][i]
        # the following path is for above 0.37 test errors
        #path = 'Calculated Distance Matrices for KNN Above 0.20/7-dists/Taxi-Pairs'+str(pair)+'-'+metric+'.csv'
        #the following path is for 100 chosen pairs for correlation and R^2 test
        path = 'Calculated Distance Matrices for KNN for comparison with RF/Taxi/7-dists/Taxi-Pairs'+str(pair)+'-'+metric+'.csv'
        np.savetxt(path, tri, delimiter=',')
    total_time = time.time() - start_time
    return total_time

In [69]:
# For pairs with above 0.20 test errors
Metrics = ['discret_frechet', 'hausdorff', 'dtw', 'sspd', 'lcss', 'edr', 'erp']

#tdist.pdist(data[pairs_20[5][1]][:20], 'edr', eps=1e-10) 
eps_edr_list = [0.005] * len(pairs_20)

#tdist.pdist(data[pairs_20[5][1]][:20], 'lcss', eps=1e-1) 
eps_lcss_list = [0.005] * len(pairs_20)

for i in range(len(pairs_20)):
    st_time = time.time()
    calculate_dists(data, metrics=Metrics, pair=pairs_20[i], eps_edr=eps_edr_list[i], 
                    eps_lcss=eps_lcss_list[i])
    print(f"time for {i}-th pair: {time.time() - st_time}")

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/traj_dist-1.1-py3.7-macosx-10.9-x86_64.egg/traj_dist/distance.py:552: UserWarning: g parameter should be specified for metric erp. Default is 
  warnings.warn("g parameter should be specified for metric erp. Default is ")


time for 0-th pair: 12.827032804489136
time for 1-th pair: 20.09112811088562
time for 2-th pair: 58.77380704879761
time for 3-th pair: 33.91453790664673
time for 4-th pair: 28.092402935028076
time for 5-th pair: 33.20237898826599
time for 6-th pair: 24.031161069869995
time for 7-th pair: 36.922812938690186
time for 8-th pair: 24.39557695388794
time for 9-th pair: 6.287843942642212
time for 10-th pair: 33.6537401676178
time for 11-th pair: 18.513312101364136
time for 12-th pair: 25.821343898773193
time for 13-th pair: 9.145843982696533
time for 14-th pair: 27.76181983947754
time for 15-th pair: 9.02119493484497
time for 16-th pair: 16.430612087249756
time for 17-th pair: 18.670878887176514
time for 18-th pair: 11.83214521408081
time for 19-th pair: 31.14607000350952
time for 20-th pair: 22.776530265808105
time for 21-th pair: 21.044973850250244
time for 22-th pair: 39.224817991256714
time for 23-th pair: 20.07357406616211
time for 24-th pair: 35.29866290092468
time for 25-th pair: 7.691

In [45]:
def KNN_with_dists(n_1, n_2, dists_names, paths_to_dists):
    '''path example: '/content/gdrive/My Drive/traj-dist/Calculated Distance Matrices (car-bus)/sspd.csv'
       dists_names: a list of distance names
       paths_to_dists: the paths list to the corresponding distancas (each path 
                       points out to the corresponding distance matrix)
       n_1: len(data_1)
       n_2: len(data_2)
       dist_name: the name of distance used to calculate sitance matrix 
       (the name is taken from a list above called metrics)'''

    train_errors = np.zeros(len(dists_names))
    test_errors = np.zeros(len(dists_names))

    I_1, J_1, y_train_1, y_test_1 = train_test_split(np.arange(n_1), 
                                                np.ones(n_1), test_size=0.3)
    I_2, J_2, y_train_2, y_test_2 = train_test_split(np.arange(n_1, n_1+n_2), 
                                                np.ones(n_2), test_size=0.3)
    labels = np.array([1]*n_1 + [0] * n_2)
    I = np.concatenate((I_1, I_2), 0)
    np.random.shuffle(I)
    J = np.concatenate((J_1, J_2), 0)
    np.random.shuffle(J)

    for i in range(len(dists_names)):

        dist_matrix = np.array(pd.read_csv(paths_to_dists[i], header=None))

        D_train = dist_matrix[I][:, I]
        D_test = dist_matrix[J][:,I]
        train_labels = labels[I]
        test_labels = labels[J]

        clf = KNeighborsClassifier(n_neighbors=5, metric='precomputed')
        
        #Train the model using the training sets
        clf.fit(D_train, list(train_labels))

        #Predict labels for train dataset
        train_pred = clf.predict(D_train)
        train_errors[i] = sum(train_labels != train_pred)/len(I)
        
        #Predict labels for test dataset
        test_pred = clf.predict(D_test)
        test_errors[i] = sum((test_labels != test_pred))/len(J)
        
    return train_errors, test_errors

In [46]:
def KNN_average_error_7_dists(data1, data2, num_trials, dists_names, paths_to_dists):

    '''dists_names: a list of distance names
       paths_to_dists: the paths list to the corresponding distancas (each path 
                       points out to the corresponding distance matrix)'''

    Start_time = time.time()

    train_errors = np.zeros((num_trials, len(dists_names)))
    test_errors = np.zeros((num_trials, len(dists_names)))

    for i in range(num_trials):
        tr_errors, ts_errors = KNN_with_dists(len(data1), len(data2), dists_names, paths_to_dists)
        train_errors[i] = tr_errors
        test_errors[i] = ts_errors

    train_error = np.mean(train_errors, axis=0)
    test_error = np.mean(test_errors, axis=0)
    std_test_error = np.std(test_errors, axis=0)

    Dict = {}
    for i in range(len(dists_names)):
        Dict[i+1] = [f"KNN with {dists_names[i]}", 
                     np.round(train_error[i], decimals = 4), 
                     np.round(test_error[i], decimals = 4), 
                     np.round(std_test_error[i], decimals = 4)]

    df = pd.DataFrame.from_dict(Dict, orient='index', columns=['Classifier',
                                'Train Error', 'Test Error', 'std'])
    print(colored(f"num_trials = {num_trials}", "blue"))
    print(colored(f'total time = {time.time() - Start_time}', 'green'))

    return [df, train_errors, test_errors, train_error, test_error, 
            np.median(test_errors, axis=0), np.std(test_errors, axis=0)]

In [302]:
# For 100 pairs
Metrics = ['discret_frechet', 'hausdorff', 'dtw', 'sspd', 'lcss', 'edr', 'erp']

#tdist.pdist(data[pairs_100[5][1]][:10], 'edr', eps=1e-10) 
#tdist.pdist(data[pairs_100[5][1]][:10], 'lcss', eps=0.05) 

for i in range(len(pairs_100)):
    st_time = time.time()
    calculate_dists(data, metrics=Metrics, pair=pairs_100[i], eps_edr=1e-10, eps_lcss=0.05)
    print(f"time for {i}-th pair: {time.time() - st_time}")

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/traj_dist-1.1-py3.7-macosx-10.9-x86_64.egg/traj_dist/distance.py:552: UserWarning: g parameter should be specified for metric erp. Default is 
  warnings.warn("g parameter should be specified for metric erp. Default is ")


time for 0-th pair: 19.202829122543335
time for 1-th pair: 11.69206690788269
time for 2-th pair: 19.121741771697998
time for 3-th pair: 27.424561977386475
time for 4-th pair: 36.33103108406067
time for 5-th pair: 56.85395884513855
time for 6-th pair: 28.69072198867798
time for 7-th pair: 12.410237312316895
time for 8-th pair: 14.80678677558899
time for 9-th pair: 54.346762895584106
time for 10-th pair: 31.288318157196045
time for 11-th pair: 26.365992069244385
time for 12-th pair: 31.905056953430176
time for 13-th pair: 15.700203895568848
time for 14-th pair: 8.747708082199097
time for 15-th pair: 22.919544219970703
time for 16-th pair: 22.293619871139526
time for 17-th pair: 35.88955092430115
time for 18-th pair: 29.685508012771606
time for 19-th pair: 7.9671311378479
time for 20-th pair: 23.17397379875183
time for 21-th pair: 9.926188707351685
time for 22-th pair: 5.839998960494995
time for 23-th pair: 25.822988033294678
time for 24-th pair: 60.177436113357544
time for 25-th pair: 32

In [304]:
# the following path is for 100 chosen pairs for correlation and R^2 test
paths = []
for i in range(len(pairs_100)):
    paths_temp = []
    for j in range(len(Metrics)):
        paths_temp.append('Calculated Distance Matrices for KNN for comparison with RF/Taxi/7-dists/Taxi-Pairs'+str(pairs_100[i])+'-'+Metrics[j]+'.csv')
    paths.append(paths_temp)
    
train_test_median_std_errors_7_dists = []
for i in range(len(pairs_100)):
    print("i=", i)
    A = KNN_average_error_7_dists(data[pairs_100[i][0]], data[pairs_100[i][1]], 
                          num_trials=100, dists_names=Metrics, paths_to_dists=paths[i])
    train_test_median_std_errors_7_dists.append(A[3:])
    print(colored(f"pair: {pairs_100[i]}", 'magenta'))
    print(A[0])
    print("============================================================")

i= 0
num_trials = 100
total time = 23.134958028793335
pair: [29 63]
                 Classifier  Train Error  Test Error     std
1  KNN with discret_frechet       0.1448      0.1904  0.0307
2        KNN with hausdorff       0.1445      0.1901  0.0344
3              KNN with dtw       0.1348      0.1919  0.0390
4             KNN with sspd       0.1202      0.1771  0.0354
5             KNN with lcss       0.2532      0.3042  0.0365
6              KNN with edr       0.8121      0.8437  0.0341
7              KNN with erp       0.1024      0.1399  0.0372
i= 1
num_trials = 100
total time = 24.859258890151978
pair: [61 140]
                 Classifier  Train Error  Test Error     std
1  KNN with discret_frechet       0.2173      0.3208  0.0353
2        KNN with hausdorff       0.2045      0.3090  0.0366
3              KNN with dtw       0.2035      0.2819  0.0382
4             KNN with sspd       0.1908      0.2792  0.0413
5             KNN with lcss       0.3121      0.3586  0.0427
6        

In [305]:
train_test_median_std_errors_discret_frechet = np.array(train_test_median_std_errors_7_dists)[:,:,0]
train_test_median_std_errors_hausdorff = np.array(train_test_median_std_errors_7_dists)[:,:,1]
train_test_median_std_errors_dtw = np.array(train_test_median_std_errors_7_dists)[:,:,2]
train_test_median_std_errors_sspd = np.array(train_test_median_std_errors_7_dists)[:,:,3]
train_test_median_std_errors_lcss = np.array(train_test_median_std_errors_7_dists)[:,:,4]
train_test_median_std_errors_edr = np.array(train_test_median_std_errors_7_dists)[:,:,5]
train_test_median_std_errors_erp = np.array(train_test_median_std_errors_7_dists)[:,:,6]

In [306]:
for i in range(len(Metrics)):
    path = '/Users/hasan/Desktop/Anaconda/Research/Calculated KNN errors for comparison with RF/Taxi-Pairs'+'-'+Metrics[i]+'.csv'
    np.savetxt(path, np.array(train_test_median_std_errors_7_dists)[:,:,i], delimiter=',')


In [74]:
# the following path is for above 0.20 test errors

Metrics = ['discret_frechet', 'hausdorff', 'dtw', 'sspd', 'lcss', 'edr', 'erp']

paths = []
train_test_median_std_errors_7_dists_20 = []
for i in range(len(pairs_20)):
    paths_temp = []
    for j in range(len(Metrics)):
        paths_temp.append('Calculated Distance Matrices for KNN Above 0.20/7-dists/Taxi-Pairs'+str(pairs_20[i])+'-'+Metrics[j]+'.csv')
    paths.append(paths_temp)
    
for i in range(len(pairs_20)):
    print(colored(f"i = {i}, pair: {pairs_20[i]}", 'magenta'))
    A = KNN_average_error_7_dists(data[pairs_20[i][0]], data[pairs_20[i][1]], 
                                  num_trials=1000, dists_names=Metrics, 
                                  paths_to_dists=paths[i])
    train_test_median_std_errors_7_dists_20.append(A[3:])
    print(A[0])
    print("============================================================")

i = 0, pair: [ 61 140]
num_trials = 1000
total time = 224.079852104187
                 Classifier  Train Error  Test Error     std
1  KNN with discret_frechet       0.2159      0.3250  0.0431
2        KNN with hausdorff       0.2079      0.3069  0.0410
3              KNN with dtw       0.2003      0.2863  0.0395
4             KNN with sspd       0.1888      0.2788  0.0433
5             KNN with lcss       0.1987      0.2974  0.0411
6              KNN with edr       0.4183      0.5037  0.0435
7              KNN with erp       0.1961      0.2926  0.0445
i = 1, pair: [129 153]
num_trials = 1000
total time = 177.40057277679443
                 Classifier  Train Error  Test Error     std
1  KNN with discret_frechet       0.1983      0.2769  0.0425
2        KNN with hausdorff       0.1844      0.2795  0.0426
3              KNN with dtw       0.1953      0.2690  0.0459
4             KNN with sspd       0.1776      0.2619  0.0434
5             KNN with lcss       0.2033      0.2930  0.0418
6 

In [75]:
for i in range(len(Metrics)):
    path = 'Calculated KNN errors for comparison with RF 20/Taxi-Pairs'+'-'+Metrics[i]+'.csv'
    np.savetxt(path, np.array(train_test_median_std_errors_7_dists_20)[:,:,i], delimiter=',')


## fastdtw which is an approximate dtw

In [77]:
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

def calculate_fastdtw_dists(data1, data2, path): 
    start_time = time.time() 
    X = np.concatenate((data1, data2), 0)
    n = len(X)
    matrix = np.zeros((n, n))
    for i in range(n):
        matrix[i][i:] = [fastdtw(X[i], X[j])[0] for j in range(i, n)]
    for i in range(1, n):
        for j in range(i):
            matrix[i][j] = matrix[j][i]
    np.savetxt(path, matrix, delimiter=',')
    total_time = time.time() - start_time
    return total_time

In [273]:
# the following path is for 100 chosen pairs for correlation and R^2 test
paths = []
for i in range(len(pairs_100)): 
    paths.append('/Users/hasan/Desktop/Anaconda/Research/Calculated Distance Matrices for KNN for comparison with RF/Taxi/fastdtw/Taxi/Taxi-Pairs['+str(pairs_100[i])+']-fastdtw.csv')

for i in range(len(pairs_100)):
    calculate_fastdtw_dists(data[pairs_100[i][0]], data[pairs_100[i][1]], path=paths[i])
    

In [78]:
# the following path is for above 0.20 test errors
paths = []
for i in range(len(pairs_20)):
    paths.append('Calculated Distance Matrices for KNN Above 0.20/fastdtw/Taxi-Pairs['+str(pairs_20[i])+']-fastdtw.csv')

for i in range(len(pairs_20)):
    calculate_fastdtw_dists(data[pairs_20[i][0]], data[pairs_20[i][1]], path=paths[i])


In [66]:
def KNN_with_fastdtw(n_1, n_2, path_to_dists):
    '''n_1: len(data_1)
       n_2: len(data_2)
       dist_name: the name of distance used to calculate sitance matrix 
       (the name is taken from a list above called metrics)'''

    I_1, J_1, y_train_1, y_test_1 = train_test_split(np.arange(n_1), 
                                                np.ones(n_1), test_size=0.3)
    I_2, J_2, y_train_2, y_test_2 = train_test_split(np.arange(n_1, n_1+n_2), 
                                                np.ones(n_2), test_size=0.3)
    labels = np.array([1]*n_1 + [0] * n_2)
    I = np.concatenate((I_1, I_2), 0)
    np.random.shuffle(I)
    J = np.concatenate((J_1, J_2), 0)
    np.random.shuffle(J)
    dist_matrix = np.array(pd.read_csv(path_to_dists, header=None))

    D_train = dist_matrix[I][:, I]
    D_test = dist_matrix[J][:,I]
    train_labels = labels[I]
    test_labels = labels[J]

    clf = KNeighborsClassifier(n_neighbors=5, metric='precomputed')
    
    #Train the model using the training sets
    clf.fit(D_train, list(train_labels))

    #Predict labels for train dataset
    train_pred = clf.predict(D_train)
    train_errors = sum(train_labels != train_pred)/len(I)
    
    #Predict labels for test dataset
    test_pred = clf.predict(D_test)
    test_errors = sum((test_labels != test_pred))/len(J)
        
    return train_errors, test_errors

In [67]:
def KNN_fastdtw_average_error(data1, data2, num_trials, path_to_dists, pair):

    '''dists_names: a list of distance names
       paths_to_dists: the paths list to the corresponding distancas (each path 
                       points out to the corresponding distance matrix)'''

    Start_time = time.time()

    train_errors = np.zeros(num_trials)
    test_errors = np.zeros(num_trials)

    for i in range(num_trials):
        tr_errors, ts_errors = KNN_with_fastdtw(len(data1), len(data2), path_to_dists)
        train_errors[i] = tr_errors
        test_errors[i] = ts_errors

    train_error = np.mean(train_errors)
    test_error = np.mean(test_errors)
    std_test_error = np.std(test_errors)

    Dict = {}
    Dict[1] = [f"KNN with fastdtw pair {pair}", 
               np.round(train_error, decimals = 4), 
               np.round(test_error, decimals = 4), 
               np.round(std_test_error, decimals = 4)]

    df = pd.DataFrame.from_dict(Dict, orient='index', columns=['Classifier',
                                'Train Error', 'Test Error', 'std'])
    print(colored(f"num_trials = {num_trials}", "blue"))
    print(colored(f'total time = {time.time() - Start_time}', 'green'))

    return (df, train_errors, test_errors, train_error, test_error, 
            np.median(test_errors), np.std(test_errors))

In [ ]:
# the following path is for 100 chosen pairs for correlation and R^2 test
paths = []
for i in range(len(pairs_100)):
    paths.append('Calculated Distance Matrices for KNN for comparison with RF/Taxi/fastdtw/Taxi-Pairs['+str(pairs_100[i])+']-fastdtw.csv')

train_test_median_std_errors_fastdtw = []
for i in range(len(pairs_100)):
    print(colored(f"i = {i}, pair: {pairs_100[i]}", 'magenta'))
    A = KNN_fastdtw_average_error(data[pairs_100[i][0]], data[pairs_100[i][1]], 
                                  num_trials=100, path_to_dists=paths[i], pair=pairs_100[i])
    train_test_median_std_errors_fastdtw.append(A[3:])
    print(A[0])
    print("=====================================================================")

In [284]:
path = '/Users/hasan/Desktop/Anaconda/Research/Calculated KNN errors for comparison with RF/Taxi-Pairs-fastdtw.csv'
np.savetxt(path, np.array(train_test_median_std_errors_fastdtw), delimiter=',')


In [862]:
# test errors = np.array(train_test_median_std_errors_fastdtw)[:,1]

In [81]:
# the following path is for above 0.20 test errors
paths = []
train_test_median_std_errors_fastdtw_20 = []
for i in range(len(pairs_20)):
    paths.append('Calculated Distance Matrices for KNN Above 0.20/fastdtw/Taxi-Pairs['+str(pairs_20[i])+']-fastdtw.csv')

for i in range(len(pairs_20)):
    print(colored(f"i = {i}, pair: {pairs_20[i]}", 'magenta'))
    fastdtw_matrix = np.array(pd.read_csv(paths[i], header=None))
    A = KNN_fastdtw_average_error(data[pairs_20[i][0]], data[pairs_20[i][1]], 
                                  num_trials=1000, path_to_dists= paths[i], 
                                  pair=pairs_20[i])
    train_test_median_std_errors_fastdtw_20.append(A[3:])
    print(A[0])
    print("==================================================================")

i = 0, pair: [ 61 140]
num_trials = 1000
total time = 28.294372081756592
                        Classifier  Train Error  Test Error     std
1  KNN with fastdtw pair [ 61 140]        0.201      0.2819  0.0402
i = 1, pair: [129 153]
num_trials = 1000
total time = 23.978668928146362
                        Classifier  Train Error  Test Error    std
1  KNN with fastdtw pair [129 153]       0.1927      0.2745  0.049
i = 2, pair: [ 81 105]
num_trials = 1000
total time = 34.95513415336609
                        Classifier  Train Error  Test Error     std
1  KNN with fastdtw pair [ 81 105]       0.1808      0.2424  0.0363
i = 3, pair: [110 127]
num_trials = 1000
total time = 26.816803693771362
                        Classifier  Train Error  Test Error     std
1  KNN with fastdtw pair [110 127]       0.1607      0.2292  0.0402
i = 4, pair: [ 95 118]
num_trials = 1000
total time = 36.34725904464722
                        Classifier  Train Error  Test Error     std
1  KNN with fastdtw pair [ 

In [82]:
path = 'Calculated KNN errors for comparison with RF 20/Taxi-Pairs-fastdtw.csv'
np.savetxt(path, np.array(train_test_median_std_errors_fastdtw_20), delimiter=',')


## Soft-dtw

In [43]:
from sdtw import SoftDTW
from sdtw.distance import SquaredEuclidean

def calculate_dists_soft_dtw(data1, data2, gamma, path): 
    start_time = time.time() 
    data = np.concatenate((data1, data2), 0)
    n = len(data)
    A = []
    for i in range(n-1):
        for j in range(i+1, n):
            D = SquaredEuclidean(data[i], data[j])
            sdtw = SoftDTW(D, gamma=gamma)
            A.append(sdtw.compute())
    A = np.array(A)
    tri = np.zeros((n, n))
    tri[np.triu_indices(n, 1)] = A
    for i in range(1, n):
        for j in range(i):
            tri[i][j] = tri[j][i]
    np.savetxt(path, tri, delimiter=',')
    total_time = time.time() - start_time
    return total_time

In [ ]:
# the following path is for 100 chosen pairs for correlation and R^2 test
paths = []
for i in range(len(pairs_100)): 
    paths.append('Calculated Distance Matrices for KNN for comparison with RF/Taxi/soft-dtw/Taxi/Taxi-Pairs['+str(pairs_100[i])+']-soft-dtw.csv')

for i in range(len(pairs_100)):
    calculate_dists_soft_dtw(data[pairs_100[i][0]], data[pairs_100[i][1]], 
                             gamma=1e-14, path=paths[i])
    

In [84]:
# the following path is for above 0.20 test errors
start_time = time.time()
paths = []
for i in range(len(pairs_20)):
    paths.append('Calculated Distance Matrices for KNN Above 0.20/soft-dtw/Taxi-Pairs['+str(pairs_20[i])+']-soft-dtw.csv')

for i in range(len(pairs_20)):
    calculate_dists_soft_dtw(data[pairs_20[i][0]], data[pairs_20[i][1]], 
                             gamma=1e-14, path=paths[i])
print("total time:", time.time() - start_time)

total time: 228.7001612186432


In [45]:
def KNN_with_dists_soft_dtw(n_1, n_2, path_to_dists):
    '''path example: path = '/Users/hasan/Desktop/Anaconda/Research/Calculated Distance Matrices for KNN/Characters-Pairs['+str(i)+']-soft-dtw.csv'
       path_to_dists: the path to the corresponding distance matrix
       n_1: len(data_1)
       n_2: len(data_2)'''

    I_1, J_1, y_train_1, y_test_1 = train_test_split(np.arange(n_1), 
                                                np.ones(n_1), test_size=0.3)
    I_2, J_2, y_train_2, y_test_2 = train_test_split(np.arange(n_1, n_1+n_2), 
                                                np.ones(n_2), test_size=0.3)
    labels = np.array([1] * n_1 + [0] * n_2)
    I = np.concatenate((I_1, I_2), 0)
    np.random.shuffle(I)
    J = np.concatenate((J_1, J_2), 0)
    np.random.shuffle(J)

    dist_matrix = np.array(pd.read_csv(path_to_dists,  header=None))

    D_train = dist_matrix[I][:, I]
    D_test = dist_matrix[J][:,I]
    train_labels = labels[I]
    test_labels = labels[J]

    clf = KNeighborsClassifier(n_neighbors=5, metric='precomputed')
    
    #Train the model using the training sets
    clf.fit(D_train, list(train_labels))

    #Predict labels for train dataset
    train_pred = clf.predict(D_train)
    train_error = sum(train_labels != train_pred)/len(I)
    
    #Predict labels for test dataset
    test_pred = clf.predict(D_test)
    test_error = sum((test_labels != test_pred))/len(J)
        
    return train_error, test_error

In [46]:
def KNN_average_error_soft_dtw(data1, data2, num_trials, path_to_dists, pair):

    '''path_to_dists: the path to the corresponding distance matrix'''

    Start_time = time.time()

    train_errors = np.zeros(num_trials)
    test_errors = np.zeros(num_trials)

    for i in range(num_trials):
        train_errors[i], test_errors[i] = KNN_with_dists_soft_dtw(len(data1), len(data2), path_to_dists)

    Dict = {}
    Dict[1] = [f"KNN with soft-dtw for pairs {pair}", 
                    np.round(np.mean(train_errors), decimals = 4), 
                    np.round(np.mean(test_errors), decimals = 4), 
                    np.round(np.std(test_errors), decimals = 4)]

    df = pd.DataFrame.from_dict(Dict, orient='index', columns=['Classifier',
                                'Train Error', 'Test Error', 'std'])
    print(colored(f"num_trials = {num_trials}", "blue"))
    print(colored(f'total time = {time.time() - Start_time}', 'green'))

    return (df, np.mean(train_errors), np.mean(test_errors), np.median(test_errors), 
            np.std(test_errors))

In [ ]:
# the following path is for 100 chosen pairs for correlation and R^2 test
paths = []
for i in range(len(pairs_100)):
    paths.append('Calculated Distance Matrices for KNN for comparison with RF/Taxi/soft-dtw/Taxi-Pairs['+str(pairs_100[i])+']-soft-dtw.csv')

train_test_median_std_errors_soft_dtw = []
for i in range(len(pairs_100)):
    print("i=", i)
    print(colored(f"pair: {pairs_100[i]}", 'magenta'))
    A = KNN_average_error_soft_dtw(data[pairs_100[i][0]], data[pairs_100[i][1]], 
                                              num_trials=100, path_to_dists=paths[i], 
                                              pair=pairs_100[i])
    train_test_median_std_errors_soft_dtw.append(A[1:])
    print(A[0])
    print("=====================================================================")

In [300]:
path = '/Users/hasan/Desktop/Anaconda/Research/Calculated KNN errors for comparison with RF/Taxi-Pairs-soft_dtw.csv'
np.savetxt(path, np.array(train_test_median_std_errors_soft_dtw), delimiter=',')


In [87]:
# test errors = np.array(train_test_median_std_errors_soft_dtw)[:,1]

In [53]:
# the following path is for above 0.20 test errors
paths = []
train_test_median_std_errors_soft_dtw_20 = []
for i in range(len(pairs_20)):
    paths.append('Calculated Distance Matrices for KNN Above 0.20/soft-dtw/Taxi-Pairs['+str(pairs_20[i])+']-soft-dtw.csv')

print(colored(f"gamma={1e-10}", 'yellow'))
for i in range(len(pairs_20)):
    print(colored(f"i = {i}, pair: {pairs_20[i]}", 'magenta'))
    A = KNN_average_error_soft_dtw(data[pairs_20[i][0]], data[pairs_20[i][1]], 
                                     num_trials=1000, path_to_dists=paths[i], 
                                     pair=pairs_20[i])
    train_test_median_std_errors_soft_dtw_20.append(A[1:])
    print(A[0])
    print("=======================================================================")

gamma=1e-10
i = 0, pair: [61, 140]
num_trials = 1000
total time = 36.533326148986816
                              Classifier  Train Error  Test Error     std
1  KNN with soft-dtw for pairs [61, 140]       0.2035      0.2923  0.0416
i = 1, pair: [129, 153]
num_trials = 1000
total time = 30.120898962020874
                               Classifier  Train Error  Test Error     std
1  KNN with soft-dtw for pairs [129, 153]       0.2036      0.2807  0.0471
i = 2, pair: [81, 105]
num_trials = 1000
total time = 43.62773013114929
                              Classifier  Train Error  Test Error     std
1  KNN with soft-dtw for pairs [81, 105]       0.1859      0.2548  0.0357
i = 3, pair: [110, 127]
num_trials = 1000
total time = 36.74912190437317
                               Classifier  Train Error  Test Error     std
1  KNN with soft-dtw for pairs [110, 127]       0.1668      0.2416  0.0402
i = 4, pair: [95, 118]
num_trials = 1000
total time = 45.80638003349304
                            

In [54]:
path = 'Calculated KNN errors for comparison with RF 20/Taxi-Pairs-soft_dtw.csv'
np.savetxt(path, np.array(train_test_median_std_errors_soft_dtw_20), delimiter=',')


## Choosing 3 pairs with best KNN test errors above 0.3

In [24]:
test_errors = np.zeros((48,12))

Metrics_all = ['discret_frechet', 'hausdorff', 'dtw', 'sspd', 'lcss', 'edr', 'erp',
               'd_Q_pi', 'dtw-tslearn', 'fastdtw', 'soft_dtw']

for i in range(7):
    path = 'Calculated KNN errors for comparison with RF 20/Taxi-Pairs'+'-'+Metrics_all[i]+'.csv'
    test_errors[:,i] = np.array(pd.read_csv(path, header=None))[:,1]
for i in range(7, len(Metrics_all)):
    path = 'Calculated KNN errors for comparison with RF 20/Taxi-Pairs'+'-'+Metrics_all[i]+'.csv'
    test_errors[:,i] = np.array(pd.read_csv(path, header=None))[:,1]

test_errors[:,11] = np.min(test_errors[:,:-1], 1)

J = np.where(test_errors[:,-1] > 0.30)[0]
print(len(J))
print(J)
test_errors = np.round(test_errors, decimals = 4)[J]

df = pd.DataFrame(data=test_errors, index=J,
                  columns= Metrics_all + ['best error'])
df

4
[13 26 33 45]


,discret_frechet,hausdorff,dtw,sspd,lcss,edr,erp,d_Q_pi,dtw-tslearn,fastdtw,soft_dtw,best error
13,0.3931,0.3865,0.3487,0.3526,0.3575,0.4967,0.3222,0.3533,0.3651,0.3455,0.3657,0.3222
26,0.3715,0.3569,0.3688,0.3193,0.3736,0.5507,0.3825,0.3810,0.3763,0.3685,0.3773,0.3193
33,0.3833,0.3606,0.3519,0.3121,0.3054,0.4654,0.4180,0.3350,0.3683,0.3549,0.3672,0.3054
45,0.3660,0.3209,0.3639,0.3320,0.3677,0.4933,0.5048,0.3588,0.3604,0.3498,0.3585,0.3209


In [28]:
pairs_20 = np.array(pairs_20)

In [230]:
# Do not run again
pairs_20[J][0], pairs_20[J][1], pairs_20[J][2]

(array([50, 94]), array([ 83, 144]), array([25, 80]))

In [234]:
#taxi_idxs[taxi_ids_selected] 
print("3 of final chosen pairs")
pairs_in_users_in_data[J] + 1

array([[3142, 6834],
       [6168, 9513],
       [1950, 5896]])

## Choosing 2 pairs with best KNN error between 0.1 and 0.2

In [75]:
test_errors = np.zeros((48,12))

Metrics_all = ['discret_frechet', 'hausdorff', 'dtw', 'sspd', 'lcss', 'edr', 'erp',
               'd_Q_pi', 'dtw-tslearn', 'fastdtw', 'soft_dtw']

for i in range(7):
    path = 'Calculated KNN errors for comparison with RF 20/Taxi-Pairs'+'-'+Metrics_all[i]+'.csv'
    test_errors[:,i] = np.array(pd.read_csv(path, header=None))[:,1]
for i in range(7, len(Metrics_all)):
    path = 'Calculated KNN errors for comparison with RF 20/Taxi-Pairs'+'-'+Metrics_all[i]+'.csv'
    test_errors[:,i] = np.array(pd.read_csv(path, header=None))[:,1]

test_errors[:,11] = np.min(test_errors[:,:-1], 1)

J = np.where((test_errors[:,-1] > 0.10) & (test_errors[:,-1] < 0.20))[0]
print(len(J))
print(J)
test_errors = np.round(test_errors, decimals = 4)[J]

df = pd.DataFrame(data=test_errors, index=J,
                  columns=Metrics_all + ['best error'])
df.loc[[15, 20]]

17
[ 1  6 10 12 15 16 17 18 19 20 23 24 28 31 43 44 47]


,discret_frechet,hausdorff,dtw,sspd,lcss,edr,erp,d_Q_pi,dtw-tslearn,fastdtw,soft_dtw,best error
15,0.1836,0.1922,0.1841,0.1924,0.2108,0.4695,0.2503,0.1766,0.1798,0.1834,0.1809,0.1766
20,0.1885,0.1897,0.1801,0.1813,0.1679,0.6560,0.2573,0.1900,0.1816,0.1832,0.1822,0.1679


In [59]:
test_ers = []
stds = []

for i in range(len(Metrics_all)):
    path = 'Calculated KNN errors for comparison with RF 20/Taxi-Pairs'+'-'+Metrics_all[i]+'.csv'
    array = np.array(pd.read_csv(path, header=None))
    ts_er = np.round(np.mean(array[:, 1][[13, 33, 45, 15, 20]]), decimals=4)
    test_ers.append(ts_er)
    std = np.round(np.mean(array[:, -1][[13, 33, 45, 15, 20]]), decimals=4)
    stds.append(std)

In [68]:
print(Metrics_all)
print(test_ers)
print(stds)

['discret_frechet', 'hausdorff', 'dtw', 'sspd', 'lcss', 'edr', 'erp', 'd_Q_pi', 'dtw-tslearn', 'fastdtw', 'soft_dtw']
[0.3029, 0.29, 0.2857, 0.2741, 0.2819, 0.5162, 0.3505, 0.2827, 0.291, 0.2834, 0.2909]
[0.0427, 0.0433, 0.043, 0.0424, 0.044, 0.0443, 0.0485, 0.043, 0.0421, 0.0426, 0.0428]


In [243]:
pairs_20[J][4], pairs_20[J][9]

(array([43, 53]), array([12, 82]))

In [250]:
print("the remained 2 of final chosen pairs")
pairs_in_users_in_data[J[4]] + 1, pairs_in_users_in_data[J[9]] + 1,

the remained 2 of final chosen pairs


(array([2876, 3260]), array([1350, 5970]))

# Chosen 5 pairs

In [190]:
pairs_final = [[50, 94], [83, 144], [25, 80], [43, 53], [12, 82]]